In [28]:
import math
import multiprocessing as mg
import multiprocessing.pool
# import pys2let as ps
import random
import string
import itertools
import os

import jax
jax.config.update("jax_enable_x64", True)
import s2fft
import healpy as hp
import numpy as np
import s2wav
import s2wav
import matplotlib.pyplot as plt
%matplotlib inline 
import skyclean
from skyclean import CMB_data

In [14]:
# Doubleworker

## Loaded mw wavelet coefficient map

## Covert to mw alm space

## add zero to the mw alms  (Is it correct? or should I add zeros to the hp alm's and then convert to mw alm's)

## Convert to from mw alm to mw map      (hp)


def doubleworker_original(MW_Pix_Map):
    '''
    Input: MW_Pix_Map: list of mw maps at different scales 
    Each pixel map is a wavelet pixel map of shape (1, Lmax, 2*Lmax-1) (MW sampling, McEwen & Wiaux)
    It is the output of s2wav.analysis
    (Scale: 0, size (1, 4, 7))

    Process:
    1. Covert MW Pixel Map to MW alm space using s2fft.forward

    2. Add zero to the mw alms  (Is it correct? or should I add zeros to the hp alm's and then convert to mw alm's)
    by adding zeros to the MW alm's we are increasing the resolution of the map
    Double the rows of the mw alms, since, the number of rows represents the L (level of detail)
    
    3. Convert mw alm to mw map 
    
    '''
    MW_alm = []

    for i in range(len(MW_Pix_Map)):
        # print("Scale:",i,"original MW Pixel Map size", MW_Pix_Map[i].shape)
        
        # Use s2fft.forward to convert to mw alm space (L_max is the maxnumber of )
        MW_alm.append(s2fft.forward(MW_Pix_Map[i], L = MW_Pix_Map[i].shape[1]))
        print("Scale:",i,"alm size", MW_alm[i].shape)
    
    MW_alm_doubled = []
    
    # for i in range(len(MW_alm)):
    #     # print("Scale:",i,"original alm size", MW_alm[i].shape)
    #     padded_alm = np.zeros((MW_alm[i].shape[0]*2,MW_alm[i].shape[1]))
    #     # stored_wavelet_coeffs_alm_doubled.append(skyclean.double_resolution(stored_wavelet_coeffs_alm[i]))
    #     padded_alm[:MW_alm[i].shape[0], :] = MW_alm[i]
    #     print("Scale:",i,"padded alm size", padded_alm.shape)
    #     MW_alm_doubled.append(padded_alm)
    
    for i in range(len(MW_alm)):
        # print("Scale:",i,"original alm size", MW_alm[i].shape)
        padded_alm = np.zeros((MW_alm[i].shape[0]*2,MW_alm[i].shape[1]*2))
        # stored_wavelet_coeffs_alm_doubled.append(skyclean.double_resolution(stored_wavelet_coeffs_alm[i]))
        padded_alm[:MW_alm[i].shape[0], :MW_alm[i].shape[1]] = MW_alm[i]
        print("Scale:",i,"padded alm size", padded_alm.shape)
        MW_alm_doubled.append(padded_alm)
    
    MW_Pix_Map_doubled = []

    for i in range(len(MW_alm_doubled)):
        # print(MW_alm_doubled[i].shape[0])
        MW_Pix_Map_doubled.append(s2fft.inverse(MW_alm_doubled[i], L = MW_alm_doubled[i].shape[0]))
        print("Scale:",i,"doubled map size", MW_Pix_Map_doubled[i].shape)

    return MW_Pix_Map_doubled

## Loaded mw wavelet coefficient map
stored_wavelet_coeffs_pix = [np.load(f"../convolution/wavelet_coefficient/wav_30_{i}.npy", allow_pickle=True) for i in range(12)]
stored_scaling_coeffs_pix = np.load("../convolution/scaling_coefficient/scal_30.npy")


# print(stored_wavelet_coeffs_pix[0].shape)
stored_wavelet_coeffs_pix = stored_wavelet_coeffs_pix[:3]

wavelet_MW_Pix_Map_doubled = doubleworker(stored_wavelet_coeffs_pix)

# display(wavelet_MW_Pix_Map_doubled[0])


Scale: 0 alm size (4, 7)
Scale: 1 alm size (4, 7)
Scale: 2 alm size (8, 15)
Scale: 0 padded alm size (8, 14)
Scale: 1 padded alm size (8, 14)
Scale: 2 padded alm size (16, 30)
Scale: 0 doubled map size (8, 15)
Scale: 1 doubled map size (8, 15)
Scale: 2 doubled map size (16, 31)


In [27]:
# Doubleworker

## Loaded mw wavelet coefficient map

## Covert to mw alm space

## add zero to the mw alms  (Is it correct? or should I add zeros to the hp alm's and then convert to mw alm's)

## Convert to from mw alm to mw map      (hp)


def Single_Map_doubleworker(MW_Pix_Map):
    '''
    Input: MW_Pix_Map: list of mw maps at different scales 
    Each pixel map is a wavelet pixel map of shape (1, Lmax, 2*Lmax-1) (MW sampling, McEwen & Wiaux)
    It is the output of s2wav.analysis
    (Scale: 0, size (1, 4, 7))

    Process:
    1. Covert MW Pixel Map to MW alm space using s2fft.forward

    2. Add zero to the mw alms  (Is it correct? or should I add zeros to the hp alm's and then convert to mw alm's)
    by adding zeros to the MW alm's we are increasing the resolution of the map
    Double the rows of the mw alms, since, the number of rows represents the L (level of detail)
    
    3. Convert mw alm to mw map 
    
    '''
    print("original Pixel Map size", MW_Pix_Map.shape)
    MW_alm = s2fft.forward(MW_Pix_Map, L = MW_Pix_Map.shape[1])
    print("original alm size", MW_alm.shape)
    
   
    # print("Scale:",i,"original alm size", MW_alm[i].shape)
    padded_alm = np.zeros((MW_alm.shape[0]*2,MW_alm.shape[1]*2))
    # stored_wavelet_coeffs_alm_doubled.append(skyclean.double_resolution(stored_wavelet_coeffs_alm[i]))
    padded_alm[:MW_alm.shape[0], :MW_alm.shape[1]] = MW_alm
    print("padded alm size", padded_alm.shape)
    MW_alm_doubled = padded_alm
    
    MW_Pix_Map_doubled = s2fft.inverse(MW_alm_doubled, L = MW_alm_doubled.shape[0])
    print("Scale:","doubled map size", MW_Pix_Map_doubled.shape)

    return MW_Pix_Map_doubled

## Loaded mw wavelet coefficient map
stored_wavelet_coeffs_pix = [np.load(f"../convolution/wavelet_coefficient/wav_30_{i}.npy", allow_pickle=True) for i in range(12)]
stored_scaling_coeffs_pix = np.load("../convolution/scaling_coefficient/scal_30.npy")


# print(stored_wavelet_coeffs_pix[0].shape)
stored_wavelet_coeffs_pix = stored_wavelet_coeffs_pix[:3]

wavelet_MW_Pix_Map_doubled = Single_Map_doubleworker(stored_wavelet_coeffs_pix[0])

# display(wavelet_MW_Pix_Map_doubled[0])
# Why oen dimension is reduced?
# Is it the spin?
print(wavelet_MW_Pix_Map_doubled.shape)


original Pixel Map size (1, 4, 7)
original alm size (4, 7)
padded alm size (8, 14)
Scale: doubled map size (8, 15)
(8, 15)


In [7]:
a = np.array([[1,2],[3,4]])
b = np.array([[5,6],[7,8]])
# display(a)
# display(b)
# display(np.multiply(a,b))

In [ ]:
display(np.multiply(wavelet_MW_Pix_Map_doubled[0],wavelet_MW_Pix_Map_doubled[1]))

In [31]:
def ILC(MW_Pix_Map):

    
    Current_Wavelet_Map = MW_Pix_Map
    
    # Define the size of the smoothing beam 
    # 1200 pixels, size of the sphere
    nsamp = 1200.0
    lmax_at_scale_j = Current_Wavelet_Map.shape[0]
    npix = hp.nside2npix(1<<(int(0.5*lmax_at_scale_j)-1).bit_length())
    # (int(0.5*scale_lmax)-1).bit_length() calculates the number of bits necessary to represent the integer int(0.5*scale_lmax)-1 in binary.
    # 1 << (int(0.5*scale_lmax)-1).bit_length() performs a bitwise left shift, essentially calculating 2^(number of bits).
    scale_fwhm = 4.0 * math.sqrt(nsamp / npix)
    # for high resolution maps, it is still the same number pixels sampled by the actual range is smaller.
    # the beam will become very narrow.
    print(lmax_at_scale_j,npix, scale_fwhm)





    


    

# ILC(wavelet_MW_Pix_Map_doubled)

# print(wavelet_MW_Pix_Map_doubled.shape)

In [ ]:
def s2let_ilc(mapsextra): #mapsextra = (j,n)
    print "Running S2LET ILC on wavelet scale", mapsextra[0], "/", jmax, "direction", mapsextra[1]+1, "/", ndir, "\n"

    if mapsextra[0] >= 0: #Wavelet scales
        scale_lmax = wav_bandlims[mapsextra[0]]
    else: #Scaling function
        scale_lmax = int(scal_bandlims)
    smoothing_lmax = 2*(scale_lmax-1)+1

    #Doubling lmax for input maps with zero-padding
    #Serial version
    '''mapsdouble = np.zeros((nrows,ps.mw_size(smoothing_lmax)),dtype=np.complex128) #Pre-allocate array
    for i in xrange(nrows):
        mapsdouble[i,:] = doubleworker((mapsextra[0][i],mapsextra[1],smoothing_lmax,mapsextra[2]))'''
    #Parallel version
    mapsextra2 = [(mapsextra[0],mapsextra[1],i,scale_lmax,smoothing_lmax) for i in xrange(nmaps)]
    print "Forming pool"
    pool2 = mg.Pool(nprocess2)
    print "Farming out workers to run doubling function"
    double_output = pool2.map(doubleworker,mapsextra2)
    print "Have returned from doubling workers\n"
    pool2.close()
    pool2.join()
    del pool2

    #Calculate scale_fwhm for smoothing kernel
    nsamp = 1200.
    npix = hp.nside2npix(1<<(int(0.5*scale_lmax)-1).bit_length()) #Equivalent no. HEALPIX pixels
    scale_fwhm = 4. * mh.sqrt(nsamp / npix)
    
    #TESTING larger covariance kernel
    scale_fwhm = 15.*scale_fwhm
    
    #Smooth covariance matrices
    #Serial version
    '''Rsmoothflat = np.zeros_like(Rflat) #Pre-allocate array
    for i in xrange(nindepelems):
        Rsmoothflat[i,:] = smoothworker((Rflat[i],smoothing_lmax,mapsextra[2],gausssmooth,mapsextra[1],mapsextra[3],i,mapsextra[4]))
    del Rflat'''
    #Parallel version
    nindepelems = int(nmaps*(nmaps+1)*.5) #No. indep. elements in symmetric covariance matrix
    Rextra = [None]*nindepelems
    k=0
    for i in xrange(nmaps):
        for j in xrange(i+1):
            Rextra[k] = (mapsextra[0],mapsextra[1],i,j,smoothing_lmax,scale_fwhm)
            k+=1
    print "Forming pool"
    pool3 = mg.Pool(nprocess3)
    print "Farming out workers to run smoothing function"
    R_output = pool3.map(smoothworker,Rextra)
    print "Have returned from smoothing workers\n"
    pool3.close()
    pool3.join()
    del pool3

    #Load R maps and form matrices
    print "Pre-allocating memory for complete covariance tensor\n"
    Rsmooth = np.zeros((ps.mw_size(smoothing_lmax),nmaps,nmaps),dtype=np.float64) #Pre-allocate array
    for i in xrange(nmaps):
        for j in xrange(i+1):
            if mapsextra[0] >= 0: #Wavelet scales
                R_fits = wav_outfits_root + '_' + wavparam_code + str(mapsextra[0]) + '_n' + str(mapsextra[1]+1) + '_Rsmooth' + str(i + 9 - nmaps) + str(j + 9 - nmaps) +'.npy'
            else: #Scaling function
                R_fits = scal_outfits[:-4] + '_Rsmooth' + str(i + 9 - nmaps) + str(j + 9 - nmaps) +'.npy'
            Rsmooth[:,i,j] = np.load(R_fits)
            if i != j:
                Rsmooth[:,j,i] = Rsmooth[:,i,j]

    #Compute inverse covariance matrices
    print "Calculating inverse covariance matrices\n"
    Rinv = np.linalg.inv(Rsmooth) #Parallel vers. slower!?- LARGEST MEMORY COST: 2*9*9*(8000^2)*complex128=0.2TB
    del Rsmooth

    #Compute weights vectors (at each pixel)
    wknumer = np.sum(Rinv,axis=-1)
    del Rinv
    wkdenom = np.sum(wknumer,axis=-1)
    wk = wknumer / wkdenom[:,None]
    del wknumer,wkdenom

    #Map loading within sub-process
    mapsdouble = np.zeros((len(wk),len(wk[0])),dtype=np.float64) #Pre-allocate array
    for i in xrange(nmaps):
        if mapsextra[0] >= 0: #Wavelet scales
            wav_fits = wav_fits_root[i + 9 - nmaps] + '_' + wavparam_code + str(mapsextra[0]) + '_n' + str(mapsextra[1]+1) + '_double.npy'
        else: #Scaling function
            wav_fits = scal_fits[i + 9 - nmaps][:-4] + '_double.npy'
        mapsdouble[:,i] = np.real(np.load(wav_fits,mmap_mode='r')) #Throw away zero imaginary part

    #Dot weights with maps (at each small pixel) - at double l(j)
    finalmap = np.sum(np.multiply(wk,mapsdouble),axis=-1) + 0.j #Add back in zero imaginary part
    del wk,mapsdouble
    
    #Downgrade resolution of MW maps
    print "Downgrading resolution of CMB wavelet map"
    finalmapalms = ps.lm2lm_hp(ps.map2alm_mw(finalmap,smoothing_lmax,spin),smoothing_lmax) #Come out in MW order - so converted to HPX order
    del finalmap
    if mapsextra[0] >= 0: #Wavelet scales
        alms_fname = wav_outfits_root + '_' + wavparam_code + str(mapsextra[0]) + '_n' + str(mapsextra[1]+1) + '_alms.fits'
    else: #Scaling function
        alms_fname = scal_outfits[:-4] + '_alms.fits'
    hp.write_alm(alms_fname,finalmapalms,lmax=scale_lmax-1,mmax=scale_lmax-1)
    del finalmapalms
    finalmapalmstruncate = hp.read_alm(alms_fname)
    finalmaphalf = ps.alm2map_mw(ps.lm_hp2lm(finalmapalmstruncate,scale_lmax),scale_lmax,spin)
    del finalmapalmstruncate
    
    #Saving output map
    if mapsextra[0] >= 1: #0: #Wavelet scales
        wav_outfits = wav_outfits_root + '_' + wavparam_code + str(mapsextra[0]) + '_n' + str(mapsextra[1]+1) + '.npy'
    elif mapsextra[0] == 0: #FOR NEW SCALING FUNC
        wav_outfits = wav_outfits_0 + '_' + wavparam_code + str(mapsextra[0]) + '_n' + str(mapsextra[1]+1) + '.npy'
    else: #Scaling function
        wav_outfits = scal_outfits[:-4] + '.npy'
    np.save(wav_outfits,finalmaphalf)
    del finalmaphalf

    return 0
